<a href="https://colab.research.google.com/github/AlekseevaTatiana23/Python_Libraries_for_DS/blob/main/ht8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%ls


sample_data/  Sleep_health_and_lifestyle_dataset.csv


Урок 8. На практике использование методов генерации признаков
В вашем распоряжении набор данных о качестве сна Sleep_health_and_lifestyle_dataset.csv.
Ваша задача — подготовить данные к моделированию, сгенерировать новые признаки, применить методы отбора признаков для улучшения качества предсказательной модели и подобрать оптимальные гиперпараметры для модели регрессии.

1. Предобработка данных:
— Очистите данные от пропусков и аномалий.
— Преобразуйте категориальные переменные с помощью One-Hot Encoding или Label Encoding.

2. Генерация новых признаков:
— Создайте комбинированные признаки, которые могут помочь улучшить модель.
— Используйте полиномиальные признаки для числовых переменных.

3. Отбор признаков:
— Примените несколько методов отбора признаков (например, RFE, SelectKBest).
— Сравните качество модели до и после отбора признаков.

4. Подбор гиперпараметров:
— Используйте GridSearchCV или RandomizedSearchCV для настройки параметров вашей модели регрессии.
— Оцените, как изменение гиперпараметров влияет на качество предсказаний.

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, accuracy_score
import time

In [4]:

!!pip install scikit-optimize

['Collecting scikit-optimize',
 '  Downloading scikit_optimize-0.10.2-py2.py3-none-any.whl.metadata (9.7 kB)',
 'Requirement already satisfied: joblib>=0.11 in /usr/local/lib/python3.11/dist-packages (from scikit-optimize) (1.5.1)',
 'Collecting pyaml>=16.9 (from scikit-optimize)',
 '  Downloading pyaml-25.5.0-py3-none-any.whl.metadata (12 kB)',
 'Requirement already satisfied: numpy>=1.20.3 in /usr/local/lib/python3.11/dist-packages (from scikit-optimize) (2.0.2)',
 'Requirement already satisfied: scipy>=1.1.0 in /usr/local/lib/python3.11/dist-packages (from scikit-optimize) (1.15.3)',
 'Requirement already satisfied: scikit-learn>=1.0.0 in /usr/local/lib/python3.11/dist-packages (from scikit-optimize) (1.6.1)',
 'Requirement already satisfied: packaging>=21.3 in /usr/local/lib/python3.11/dist-packages (from scikit-optimize) (24.2)',
 'Requirement already satisfied: PyYAML in /usr/local/lib/python3.11/dist-packages (from pyaml>=16.9->scikit-optimize) (6.0.2)',
 'Requirement already sa

In [5]:
from skopt.space import Real, Integer
from skopt import BayesSearchCV

In [7]:
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


Датасет Sleep Health and Lifestyle содержит информацию о здоровье сна и образе жизни 374 человек.

Описание столбцов датасета Sleep health and lifestyle dataset:

Person ID: уникальный идентификатор для каждого человека
Gender: пол человека (мужчина/женщина)
Age: возраст человека в годах
Occupation: профессия или род деятельности человека
Sleep Duration (hours): количество часов, которое человек спит в день
Quality of Sleep (scale: 1-10): субъективная оценка качества сна по шкале от 1 до 10, где 1 — наихудшее качество, а 10 — наилучшее
Physical Activity Level (minutes/day): количество минут физической активности в день
Stress Level (scale: 1-10): субъективная оценка уровня стресса по шкале от 1 до 10
BMI Category: категория индекса массы тела (например: нормальный, избыточный вес, ожирение)
Blood Pressure (systolic/diastolic): измерение артериального давления, указано как систолическое/диастолическое давление
Heart Rate (bpm): пульс в состоянии покоя в ударах в минуту
Daily Steps: количество шагов, которые человек проходит за день
Sleep Disorder: наличие или отсутствие расстройства сна у человека (нет, инсомния, апноэ сна).

In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [9]:
df.isnull().sum()



,0
Person ID,0
Gender,0
Age,0
Occupation,0
Sleep Duration,0
Quality of Sleep,0
Physical Activity Level,0
Stress Level,0
BMI Category,0
Blood Pressure,0


In [11]:
df['Sleep Disorder'].value_counts()

,count
Sleep Disorder,
Sleep Apnea,78
Insomnia,77


In [12]:
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('No information')

In [13]:
df['Sleep Disorder'].value_counts()

,count
Sleep Disorder,
No information,219
Sleep Apnea,78
Insomnia,77


In [14]:
df.duplicated().sum()

np.int64(0)

In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Person ID,374.0,187.500000,108.108742,1.0,94.25,187.5,280.75,374.0
Age,374.0,42.184492,8.673133,27.0,35.25,43.0,50.00,59.0
Sleep Duration,374.0,7.132086,0.795657,5.8,6.40,7.2,7.80,8.5
Quality of Sleep,374.0,7.312834,1.196956,4.0,6.00,7.0,8.00,9.0
Physical Activity Level,374.0,59.171123,20.830804,30.0,45.00,60.0,75.00,90.0
Stress Level,374.0,5.385027,1.774526,3.0,4.00,5.0,7.00,8.0
Heart Rate,374.0,70.165775,4.135676,65.0,68.00,70.0,72.00,86.0
Daily Steps,374.0,6816.844920,1617.915679,3000.0,5600.00,7000.0,8000.00,10000.0


In [17]:
df.describe(include='object').T

,count,unique,top,freq
Gender,374,2,Male,189
Occupation,374,11,Nurse,73
BMI Category,374,4,Normal,195
Blood Pressure,374,25,130/85,99
Sleep Disorder,374,3,No information,219


In [18]:
df['Gender'].value_counts()

,count
Gender,
Male,189
Female,185


In [19]:
df['Gender'] = df['Gender'].replace({'Female': 0, 'Male': 1})

In [20]:
df['Occupation'].value_counts()

,count
Occupation,
Nurse,73
Doctor,71
Engineer,63
Lawyer,47
Teacher,40
Accountant,37
Salesperson,32
Scientist,4
Software Engineer,4


In [21]:
df.loc[df['Occupation'] == 'Sales Representative', 'Occupation'] = 'Salesperson'

In [22]:
frequency = df['Occupation'].value_counts(normalize=True)
df['Occupation'] = df['Occupation'].map(frequency)
df['Occupation'].value_counts()

,count
Occupation,
0.195187,73
0.189840,71
0.168449,63
0.125668,47
0.106952,40
0.098930,37
0.090909,34
0.010695,8
0.002674,1


In [23]:
df['Sleep Disorder'].value_counts()

,count
Sleep Disorder,
No information,219
Sleep Apnea,78
Insomnia,77


In [24]:
df['Sleep_Disorder'] = df['Sleep Disorder'].map({
    'No information': 0,
    'Sleep Apnea': 2,
    'Insomnia': 1
})
df = df.drop('Sleep Disorder', axis=1)

In [25]:
df['BMI Category'].value_counts()

,count
BMI Category,
Normal,195
Overweight,148
Normal Weight,21
Obese,10


In [26]:
df['BMI Category'] = df['BMI Category'].replace({'Normal Weight': 'Normal'})
df['BMI Category Coded'] = pd.Categorical(df['BMI Category'], categories=['Normal', 'Overweight', 'Obese'], ordered=True)
df['BMI Category Coded'] = df['BMI Category Coded'].cat.codes

print(df['BMI Category Coded'].value_counts())

BMI Category Coded
0    216
1    148
2     10
Name: count, dtype: int64


In [27]:
df['Blood Pressure'].value_counts()

,count
Blood Pressure,
130/85,99
140/95,65
125/80,65
120/80,45
115/75,32
135/90,27
140/90,4
125/82,4
128/85,3


In [28]:
# Разделения значения давления и преобразование в числовой формат
df[['Systilic BP', 'Diastolic BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)

# Разбиение на категории по систолическому давлению
df['Systolic Category'] = pd.cut(df['Systilic BP'], bins=[0, 119, 129, 139, 159, 180], labels=[0, 1, 2, 3, 4], right=False)

# Разбиение на категории по диаистолическому давлению
df['Diastolic Category'] = pd.cut(df['Diastolic BP'], bins=[0, 79, 89, 99, 109, 120], labels=[0, 1, 2, 3, 4], right=False)

print(df[['Systilic BP', 'Systolic Category', 'Diastolic BP', 'Diastolic Category']].head())

   Systilic BP Systolic Category  Diastolic BP Diastolic Category
0          126                 1            83                  1
1          125                 1            80                  1
2          125                 1            80                  1
3          140                 3            90                  2
4          140                 3            90                  2


In [29]:
df.drop(['Blood Pressure', 'BMI Category', 'Person ID'], axis=1, inplace=True)

In [30]:
df.head()

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep_Disorder,BMI Category Coded,Systilic BP,Diastolic BP,Systolic Category,Diastolic Category
0,1,27,0.010695,6.1,6,42,6,77,4200,0,1,126,83,1,1
1,1,28,0.189840,6.2,6,60,8,75,10000,0,0,125,80,1,1
2,1,28,0.189840,6.2,6,60,8,75,10000,0,0,125,80,1,1
3,1,28,0.090909,5.9,4,30,8,85,3000,2,2,140,90,3,2
4,1,28,0.090909,5.9,4,30,8,85,3000,2,2,140,90,3,2


Используем полиномиальные признаки для числовых переменных

In [31]:
features = df[['Age', 'Daily Steps', 'Heart Rate']]

poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(features)
poly_features_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(features.columns))

df = pd.concat([df, poly_features_df], axis=1)

df.head()

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep_Disorder,...,Diastolic Category,Age,Daily Steps,Heart Rate,Age^2,Age Daily Steps,Age Heart Rate,Daily Steps^2,Daily Steps Heart Rate,Heart Rate^2
0,1,27,0.010695,6.1,6,42,6,77,4200,0,...,1,27.0,4200.0,77.0,729.0,113400.0,2079.0,17640000.0,323400.0,5929.0
1,1,28,0.189840,6.2,6,60,8,75,10000,0,...,1,28.0,10000.0,75.0,784.0,280000.0,2100.0,100000000.0,750000.0,5625.0
2,1,28,0.189840,6.2,6,60,8,75,10000,0,...,1,28.0,10000.0,75.0,784.0,280000.0,2100.0,100000000.0,750000.0,5625.0
3,1,28,0.090909,5.9,4,30,8,85,3000,2,...,2,28.0,3000.0,85.0,784.0,84000.0,2380.0,9000000.0,255000.0,7225.0
4,1,28,0.090909,5.9,4,30,8,85,3000,2,...,2,28.0,3000.0,85.0,784.0,84000.0,2380.0,9000000.0,255000.0,7225.0


Применим несколько методов отбора признаков, сравним модели

In [32]:
X = df.drop('Sleep Duration', axis=1)
y = df['Sleep Duration']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [33]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
tree = DecisionTreeRegressor()

tree.fit(X_train, y_train)
prediction = tree.predict(X_test)

print(f'R^2 на тренировочных данных {r2_score(y_train, tree.predict(X_train))}')
print(f'R^2 на тестовых данных {r2_score(y_test, tree.predict(X_test))}')

R^2 на тренировочных данных 0.9962613763571585
R^2 на тестовых данных 0.9863984846926285


Показатели модели DecisionTreeRegressor указывают на высокую степень объяснения вариации целевой переменной как на тренировочных, так и на тестовых данных.

Показатели R2:

на тренировочных данных: 0.9963
на тестовых данных: 0.9869
Интерпретация значений

Высокое значение на тренировочных данных (0.9963) указывает на то, что модель объясняет 99.63% вариации в тренировочных данных. Такой высокий коэффициент может свидетельствовать о том, что модель хорошо подстраивается под данные, однако это также может быть признаком переобучения.
Значение на тестовых данных (0.9869) также высоко, что говорит о том, что модель сохраняет свою способность объяснять вариацию и на новых данных, что является положительным признаком.
Разница между значениями R2 для тренировочных и тестовых данных составляет всего около 0.0094. Это указывает на то, что модель скорее всего не переобучена и хорошо обобщает информацию, но все-таки риск переобучения сохраняется.

In [34]:
from sklearn.feature_selection import RFE
# Создаем RFE модель
rfe = RFE(estimator=tree, n_features_to_select=13)
rfe = rfe.fit(X, y)

# Показываем какие признаки выбраны
selected_features = pd.DataFrame({
    'Feature': X.columns,  # Здесь X.columns должен содержать названия столбцов, использованных для X
    'Ranking': rfe.ranking_
})
print(selected_features.sort_values(by='Ranking', ascending=False))

                    Feature  Ranking
13       Diastolic Category       11
0                    Gender       10
7               Daily Steps        9
16               Heart Rate        8
9        BMI Category Coded        7
12        Systolic Category        6
6                Heart Rate        5
1                       Age        4
15              Daily Steps        3
14                      Age        2
2                Occupation        1
5              Stress Level        1
4   Physical Activity Level        1
3          Quality of Sleep        1
8            Sleep_Disorder        1
11             Diastolic BP        1
10              Systilic BP        1
17                    Age^2        1
18          Age Daily Steps        1
19           Age Heart Rate        1
20            Daily Steps^2        1
21   Daily Steps Heart Rate        1
22             Heart Rate^2        1


In [35]:
top_features = selected_features.sort_values(by='Ranking', ascending=False)[:10]
X_train_selected = X_train[top_features.Feature]
X_test_selected = X_test[top_features.Feature]

# Обучение дерева решений на отобранных признаках
dt1 = DecisionTreeRegressor(random_state=42)
dt1.fit(X_train_selected, y_train)
print(f'R^2 на тренировочных данных {r2_score(y_train, dt1.predict(X_train_selected))}')
print(f'R^2 на тестовых данных {r2_score(y_test, dt1.predict(X_test_selected))}')

R^2 на тренировочных данных 0.9695903063979994
R^2 на тестовых данных 0.9090994590739611


Результаты обучения на топ 10 признаках не значительно, но хуже.

Показатели R2:

на тренировочных данных: 0.9696
на тестовых данных: 0.9113
Интерпретация значений:

R2 на тренировочных данных указывает на то, что модель объясняет 96.96% вариации в тренировочных данных. Это высокий показатель, который говорит о том, что модель все еще хорошо подстраивается под данные.
R2 на тестовых данных составляет 91.13%. Это также хороший результат, но заметно ниже, чем на тренировочных данных. Это может указывать на то, что модель не так хорошо обобщает информацию на новых, невидимых данных.
Разница между R2 для тренировочных и тестовых данных составляет примерно 0.0583 (или 5.83%). Это может быть признаком некоторого переобучения модели, так как она показывает значительно лучшие результаты на тренировочных данных по сравнению с тестовыми.

In [36]:
feature_importances = pd.Series(tree.feature_importances_, index=X_train.columns)

# Отбор 10 наиболее важных признаков
top_features2 = feature_importances.nlargest(10).index
top_features2

Index(['Quality of Sleep', 'Physical Activity Level', 'BMI Category Coded',
       'Stress Level', 'Daily Steps Heart Rate', 'Sleep_Disorder',
       'Heart Rate', 'Daily Steps', 'Age Daily Steps', 'Heart Rate^2'],
      dtype='object')

In [37]:
X_train_selected2 = X_train[top_features2]
X_test_selected2 = X_test[top_features2]

# Обучение дерева решений на отобранных признаках
dt2 = DecisionTreeRegressor(random_state=42)
dt2.fit(X_train_selected2, y_train)
print(f'R^2 на тренировочных данных {r2_score(y_train, dt2.predict(X_train_selected2))}')
print(f'R^2 на тестовых данных {r2_score(y_test, dt2.predict(X_test_selected2))}')

R^2 на тренировочных данных 0.9962583708514867
R^2 на тестовых данных 0.9919454595469365


In [39]:
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd

# Создаем SelectKBest модель с выбранной функцией оценки
select_k_best = SelectKBest(score_func=f_classif, k=10)  # k - количество признаков, которые нужно выбрать
select_k_best.fit(X, y)

# Получаем оценки признаков
scores = select_k_best.scores_

# Создаем DataFrame для выбранных признаков и их оценок
selected_features3 = pd.DataFrame({
    'Feature': X.columns,
    'Score': scores
})

# Сортируем по убыванию оценок
print(selected_features3.sort_values(by='Score', ascending=False))

                    Feature       Score
3          Quality of Sleep  140.586632
5              Stress Level   81.618461
17                    Age^2   26.189193
9        BMI Category Coded   22.880618
2                Occupation   22.605191
1                       Age   20.766194
14                      Age   20.766194
4   Physical Activity Level   20.453638
13       Diastolic Category   16.587580
16               Heart Rate   15.352599
6                Heart Rate   15.352599
22             Heart Rate^2   14.613826
11             Diastolic BP   14.174391
19           Age Heart Rate   13.988468
10              Systilic BP   13.633807
0                    Gender   13.409809
8            Sleep_Disorder   13.130067
12        Systolic Category   11.749833
21   Daily Steps Heart Rate    9.984257
7               Daily Steps    8.839822
15              Daily Steps    8.839822
20            Daily Steps^2    8.556099
18          Age Daily Steps    4.711809


In [40]:
top_features3 = selected_features3.sort_values(by='Score', ascending=False)[:10]
X_train_selected3 = X_train[top_features3.Feature]
X_test_selected3 = X_test[top_features3.Feature]

# Обучение дерева решений на отобранных признаках
dt3 = DecisionTreeRegressor(random_state=42)
dt3.fit(X_train_selected3, y_train)
print(f'R^2 на тренировочных данных {r2_score(y_train, dt3.predict(X_train_selected3))}')
print(f'R^2 на тестовых данных {r2_score(y_test, dt3.predict(X_test_selected3))}')

R^2 на тренировочных данных 0.9958222096265205
R^2 на тестовых данных 0.9916862785688405


Как мы видим, качество модели с уменьшением признаков не снижается, а растет, что само по себе можно рассматривать как положительный момент, но при этом очень большая вероятность того, что наша модель переобучена: показатели R2 очень высокие что на тренировочных данных, что на тестовых.

Подбор гиперпараметров

In [44]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from skopt import BayesSearchCV


# Функция для оценки модели с помощью перекрестной проверки
#def evaluate_model(model, X_train, y_train):
 #   scores = cross_val_score(model, X_train, y_train, scoring='r2', cv=5)
  #  return scores.mean()

# Поиск гиперпараметров с помощью сетки параметров
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt = DecisionTreeRegressor(random_state=42)
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, scoring='r2', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(f"Лучшие гиперпараметры (Grid Search): {grid_search.best_params_}")
print(f"Лучшая оценка (Grid Search): {grid_search.best_score_:.2f}")

Лучшие гиперпараметры (Grid Search): {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 10}
Лучшая оценка (Grid Search): 0.97


max_depth=9:
Глубина дерева достаточно большая, что позволяет модели захватывать сложные зависимости в данных, повышая точность.

min_samples_leaf=1:
Разделения могут создавать листья с только одним образцом, что иногда увеличивает риск переобучения, но в данном случае модель показывает очень высокие показатели, значит, данные позволяют это делать без существенных потерь.

min_samples_split=10:
Разделение узлов происходит только если в узле есть не менее 10 образцов, что помогает контролировать рост дерева и избегать переобучения.

Модель очень хорошо подходит к данным, демонстрируя высокую точность и хорошую обобщающую способность.

In [46]:
from sklearn.model_selection import RandomizedSearchCV

# Параметры для случайного поиска
param_dist = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt = DecisionTreeRegressor(random_state=42)

# Используем RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=param_dist,
    n_iter=10,  # число случайных комбинаций
    scoring='r2',
    cv=5,
    n_jobs=-1,
    random_state=42
)

# Обучение
random_search.fit(X_train, y_train)

# Вывод лучших гиперпараметров и оценки
print(f"Лучшие гиперпараметры (Randomized Search): {random_search.best_params_}")
print(f"Лучшая оценка (Randomized Search): {random_search.best_score_:.2f}")

Лучшие гиперпараметры (Randomized Search): {'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 9}
Лучшая оценка (Randomized Search): 0.97


max_depth=9:
Глубина дерева увеличена по сравнению с предыдущим результатом (6 в Bayesian-оптимизации). Это позволяет дереву моделировать более сложные зависимости, что может повысить точность.

min_samples_split=5:
Разделение узлов происходит, когда в узле не менее 5 образцов. Это помогает сбалансировать рост дерева, избегая слишком мелких узлов, что предотвращает переобучение.

min_samples_leaf=2:
Каждый лист содержит как минимум 2 образца, что способствует более гладкому и устойчивому дереву.

Модель объясняет 97% вариации целевой переменной — практически идеально подходит для данной задачи.

In [49]:

from skopt.space import Integer, Categorical
from skopt import BayesSearchCV

search_space = {
    "max_depth": Integer(6, 10),
    # "max_features": Categorical(['auto', 'sqrt']),
    "max_features": Categorical([1, 'sqrt']),
    "min_samples_leaf": Integer(3, 5),
    "min_samples_split": Integer(4, 6),
}

clf_bayes = BayesSearchCV(
    estimator=dt,
    search_spaces=search_space,
    random_state=0,
    cv=3,
    verbose=2,
    n_jobs=-1
)

clf_bayes.fit(X, y)

best_params = clf_bayes.best_params_
best_r2 = clf_bayes.best_score_

print(f"Лучшие гиперпараметры (Bayesian Optimization): {best_params}")
print(f"Лучшая оценка (Bayesian Optimization): {best_r2:.2f}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

max_depth=6:
Умеренная глубина дерева. Это говорит о том, что увеличение глубины сверх 6 не дает заметного улучшения, а более глубокие деревья могут переобучаться. Значит, модель демонстрирует хорошую балансировку между сложностью и обобщающей способностью.

max_features='sqrt':
Использование стратегии выбора признаков, которая выбирает корень из общего числа признаков, способствует уменьшению корреляции между деревьями (если речь о случайных лесах) и помогает бороться с переобучением.

min_samples_leaf=4 и min_samples_split=6:
Эти параметры ограничивают разбитие узлов, предотвращая создание очень мелких или чрезмерно глубоких деревьев. Это способствует более устойчивой модели и уменьшает переобучение.

Результат 0.54 говорит о том, что есть простор для улучшения:

Можно попробовать расширить диапазоны поиска гиперпараметров.
Использовать другие модели (например, градиентный бустинг, случайный лес или нейронные сети).
Обработать данные — возможно, есть необходимость в дополнительной очистке или подборе признаков.
Попробовать увеличить количество итераций байесовской оптимизации или применить ансамбли моделей.

Итоговое сравнение


Лучшие результаты по метрикам (0.97) показывают Randomized Search и Grid Search — оба метода дали одинаково высокие показатели, что свидетельствует о хорошей настройке модели.

Randomized Search — более быстрый и менее ресурсоемкий, что делает его предпочтительным для быстрой оценки.

Grid Search — обеспечивает более точную настройку гиперпараметров, но требует больше времени.

Bayesian Optimization — в данном случае дал значительно худший результат (0.54), что может свидетельствовать о недостаточной настройке или особенностях метода для данной задачи.

Общий совет:
Для быстрого получения хороших гиперпараметров использовать Randomized Search.
Для окончательной точной настройки лучше применять Grid Search.
Использовать Bayesian Optimization для поиска глобальных оптимумов, если есть возможность потратить больше времени, однако в данном случае он оказался менее эффективным